In [1]:
from __future__ import annotations

from vtkbone import vtkboneAIMReader

from bonelab.util.aim_calibration_header import get_aim_density_equation
from bonelab.util.vtk_util import vtkImageData_to_numpy
import numpy as np
from enum import Enum

import SimpleITK as sitk
import pyvista as pv
import vtk

import os

Data locations

In [2]:
sample_visualizations_dir = os.path.join(
    "/", "Users", "nathanneeteson", "Documents", "Data",
    "Images", "AutoKneeAnalysis", "overview_figure"
)

image_name = "SLTC007L"

Load the data

In [3]:
resample_factor = 2

reader = vtkboneAIMReader()
reader.DataOnCellsOff()

image_data = {}
image_data["F"] = {}
image_data["T"] = {}

for bone in ["F", "T"]:
    for image_type in ["CORT", "TRAB", "ROI_ALL"]:
        
        reader.SetFileName(os.path.join(
            sample_visualizations_dir,
            f"{image_name}_{bone}_{image_type}.AIM"
        ))
        reader.Update()
        
        image_data[bone][image_type] = pv.wrap(reader.GetOutput())
        
        
        image_data[bone][image_type] = pv.create_grid(
            image_data[bone][image_type],
            dimensions=[d//resample_factor for d in image_data[bone][image_type].dimensions]
        ).sample(image_data[bone][image_type], categorical=True, progress_bar=True)
        
   

Resampling array Data from a Passed Mesh onto Mesh: 100%|████████████████████████████████████████████[00:48<00:00]
Resampling array Data from a Passed Mesh onto Mesh: 100%|████████████████████████████████████████████[00:49<00:00]
Resampling array Data from a Passed Mesh onto Mesh: 100%|████████████████████████████████████████████[00:53<00:00]
Resampling array Data from a Passed Mesh onto Mesh: 100%|████████████████████████████████████████████[00:21<00:00]
Resampling array Data from a Passed Mesh onto Mesh: 100%|████████████████████████████████████████████[00:18<00:00]
Resampling array Data from a Passed Mesh onto Mesh: 100%|████████████████████████████████████████████[00:21<00:00]


In [4]:
progress_bar = True
roi_contours = list(np.arange(9.5,38,1))

for img in image_data.values():
    img["CORT contour"] = img["CORT"].contour([126], progress_bar=progress_bar)
    
for roi_code in range(10,18):
    image_data["F"][f"ROI_{roi_code}"] = image_data["F"]["ROI_ALL"].threshold(
        (roi_code - 0.5, roi_code + 0.5),
        progress_bar=progress_bar
    ).extract_surface(progress_bar=progress_bar)
    
for roi_code in range(30,38):
    image_data["T"][f"ROI_{roi_code}"] = image_data["T"]["ROI_ALL"].threshold(
        (roi_code - 0.5, roi_code + 0.5),
        progress_bar=progress_bar
    ).extract_surface(progress_bar=progress_bar)

Computing Contour: 100%|█████████████████████████████████████████████████████████████████████████████[00:02<00:00]
Computing Contour: 100%|█████████████████████████████████████████████████████████████████████████████[00:01<00:00]
Thresholding: 100%|██████████████████████████████████████████████████████████████████████████████████[00:15<00:00]
Extracting Surface: 100%|████████████████████████████████████████████████████████████████████████████[00:00<00:00]
Thresholding: 100%|██████████████████████████████████████████████████████████████████████████████████[00:15<00:00]
Extracting Surface: 100%|████████████████████████████████████████████████████████████████████████████[00:00<00:00]
Thresholding: 100%|██████████████████████████████████████████████████████████████████████████████████[00:15<00:00]
Extracting Surface: 100%|████████████████████████████████████████████████████████████████████████████[00:00<00:00]
Thresholding: 100%|█████████████████████████████████████████████████████████████

In [5]:
smoothing_iterations = 1000

for img in image_data.values():
    img["CORT contour smooth"] = img["CORT contour"].smooth(smoothing_iterations, progress_bar=progress_bar)
    
for i in range(8):
    image_data["F"][f"ROI_{10+i} smooth"] = image_data["F"][f"ROI_{10+i}"].smooth(
        smoothing_iterations, progress_bar=progress_bar
    )
    image_data["T"][f"ROI_{30+i} smooth"] = image_data["T"][f"ROI_{30+i}"].smooth(
        smoothing_iterations, progress_bar=progress_bar
    )

Smoothing Mesh: 100%|████████████████████████████████████████████████████████████████████████████████[01:34<00:00]
Smoothing Mesh: 100%|████████████████████████████████████████████████████████████████████████████████[00:49<00:00]
Smoothing Mesh: 100%|████████████████████████████████████████████████████████████████████████████████[00:03<00:00]
Smoothing Mesh: 100%|████████████████████████████████████████████████████████████████████████████████[00:02<00:00]
Smoothing Mesh: 100%|████████████████████████████████████████████████████████████████████████████████[00:03<00:00]
Smoothing Mesh: 100%|████████████████████████████████████████████████████████████████████████████████[00:01<00:00]
Smoothing Mesh: 100%|████████████████████████████████████████████████████████████████████████████████[00:03<00:00]
Smoothing Mesh: 100%|████████████████████████████████████████████████████████████████████████████████[00:01<00:00]
Smoothing Mesh: 100%|███████████████████████████████████████████████████████████

In [ ]:
base_dx = 30
base_dz = 8
cort_opacity = 0.6
rois_opacity = 0.8
line_width = 5
line_origin_jitter = 3

# the order of the ROIs is:
# 0 - lat shallow
# 1 - lat middle
# 2 - lat deep
# 3 - med shallow
# 4 - med middle
# 5 - med deep
# 6 - lat plate
# 7 - med plate

colors = [
    "yellow",
    "orange",
    "red",
    "yellow",
    "orange",
    "red",
    "green",
    "green"
]

dx = [
    base_dx,
    base_dx,
    base_dx,
    -base_dx,
    -base_dx,
    -base_dx,
    base_dx,
    -base_dx
]

dz = [
    0,
    base_dz,
    2*base_dz,
    0,
    base_dz,
    2*base_dz,
    0,
    0
]

line_origin_jitter = [
    0,
    2,
    -2,
    0,
    -2,
    2,
    0,
    0
]

pl = pv.Plotter(
    notebook=False, 
    lighting='three lights', 
    window_size=(1200,600),
    line_smoothing=True,
    polygon_smoothing=True,
    theme=pv.themes.DocumentTheme()
)

for img in image_data.values():
    pl.add_mesh(
        img["CORT contour smooth"],
        color="w", opacity=cort_opacity
    )

# plot inside
for i in range(8):
    pl.add_mesh(
        image_data["F"][f"ROI_{10+i} smooth"], 
        color=colors[i], opacity=rois_opacity
    )
    pl.add_mesh(
        image_data["T"][f"ROI_{30+i} smooth"], 
        color=colors[i], opacity=rois_opacity
    )

    
# plot outside
for i in range(8):
    
    pl.add_mesh(
        image_data["F"][f"ROI_{10+i} smooth"].translate(
            [1.2*dx[i],0,dz[i]]
        ), 
        color=colors[i]
    )
    b = image_data["F"][f"ROI_{10+i} smooth"].bounds
    c = [b[2*i] + (b[2*i+1] - b[2*i])/2 for i in range(3)]
    pl.add_mesh(
        pv.lines_from_points([
            [c[0]+line_origin_jitter[i], c[1], c[2]],
            [c[0]+line_origin_jitter[i], c[1], c[2]+dz[i]],
            [c[0]+1.2*dx[i], c[1], c[2]+dz[i]]
        ]),
        line_width=line_width,
        color=colors[i]
    )
    
    pl.add_mesh(
        image_data["T"][f"ROI_{30+i} smooth"].translate(
            [dx[i],0,-0.7*dz[i]]
        ), 
        color=colors[i]
    )
    b = image_data["T"][f"ROI_{30+i} smooth"].bounds
    c = [b[2*i] + (b[2*i+1] - b[2*i])/2 for i in range(3)]
    pl.add_mesh(
        pv.lines_from_points([
            [c[0]+line_origin_jitter[i], c[1], c[2]],
            [c[0]+line_origin_jitter[i], c[1], c[2]-0.7*dz[i]],
            [c[0]+dx[i], c[1], c[2]-0.7*dz[i]]
        ]),
        line_width=line_width,
        color=colors[i]
    )


pl.camera_position = "xz"
pl.camera.zoom(2)
        
pl.show()

In [7]:
pv.__version__

'0.38.6'